In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# ✅ Define the MLP Model
class MLP(nn.Module):
    def __init__(self, input_size=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.2)

        self.fc5 = nn.Linear(128, 2)  # Output size = 2 (Binary Classification)

    def forward(self, x):
        x = self.relu1(self.bn1(self.fc1(x)))
        x = self.dropout1(x)

        x = self.relu2(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        x = self.relu3(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        x = self.relu4(self.bn4(self.fc4(x)))
        x = self.dropout4(x)

        x = self.fc5(x)
        return x

# ✅ Load Dataset and Preprocess
def load_dataset(client_id):
    df = pd.read_csv("middle_aged_adults.csv")  # Make sure the dataset is present
    X = df.drop(columns=['TenYearCHD']).values  # Features
    y = df['TenYearCHD'].values  # Labels

    # ✅ Apply SMOTE for balancing
    smote = SMOTE(random_state=42 + client_id)
    X, y = smote.fit_resample(X, y)

    # ✅ Split into training and test sets (each client gets different data)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42 + client_id
    )

    # ✅ Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # ✅ Create DataLoaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

    return train_loader, test_loader, X_train_tensor.shape[1]  # Input size for model

# ✅ Initialize Client
client_id = int(input("Enter client ID (1 or 2): "))  # Manually input ID
train_loader, test_loader, input_size = load_dataset(client_id)

model = MLP(input_size=input_size)
optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

class FLClient(fl.client.NumPyClient):
    def __init__(self, client_id):
        self.client_id = client_id

    def get_parameters(self, config):
        """Return current model parameters."""
        return [val.cpu().detach().numpy() for val in model.state_dict().values()]

    def set_parameters(self, parameters):
        """Update model parameters before training."""
        state_dict = dict(zip(model.state_dict().keys(), parameters))
        model.load_state_dict({k: torch.tensor(v) for k, v in state_dict.items()}, strict=False)

    def fit(self, parameters, config):
        """Train the model for 10 local epochs and return updated parameters."""
        self.set_parameters(parameters)
        model.train()

        for epoch in range(10):  # 🔹 Train for 10 epochs before sending update
            total_loss = 0.0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            print(f"📢 [Client {self.client_id}] Epoch {epoch+1}/10, Loss: {total_loss/len(train_loader):.4f}")

        print(f"📤 Client {self.client_id} training complete. Sending updates.")
        return self.get_parameters(config), len(train_loader.dataset), {}

    def evaluate(self, parameters, config):
        """Evaluate model and return loss & accuracy."""
        self.set_parameters(parameters)
        model.eval()

        total_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                total_loss += loss.item()
                correct += (outputs.argmax(dim=1) == y_batch).sum().item()
                total += y_batch.size(0)

        accuracy = correct / total if total > 0 else 0.0
        print(f"✅ Client {self.client_id} Evaluation: Loss={total_loss:.4f}, Accuracy={accuracy:.4f}")
        return float(total_loss), len(test_loader.dataset), {"accuracy": accuracy}

# ✅ Start the client
fl.client.start_numpy_client(server_address="127.0.0.1:8081", client=FLClient(client_id))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# ✅ Define the MLP Model
class MLP(nn.Module):
    def __init__(self, input_size=10):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)

        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(0.2)

        self.fc5 = nn.Linear(128, 2)  # Output size = 2 (Binary Classification)

    def forward(self, x):
        x = self.relu1(self.bn1(self.fc1(x)))
        x = self.dropout1(x)

        x = self.relu2(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        x = self.relu3(self.bn3(self.fc3(x)))
        x = self.dropout3(x)

        x = self.relu4(self.bn4(self.fc4(x)))
        x = self.dropout4(x)

        x = self.fc5(x)
        return x

# ✅ Load Dataset and Preprocess
def load_dataset():
    df = pd.read_csv("middle_aged_adults.csv")  # Make sure the dataset is present
    X = df.drop(columns=['TenYearCHD']).values  # Features
    y = df['TenYearCHD'].values  # Labels

    # ✅ Apply SMOTE for balancing
    smote = SMOTE(random_state=42)
    X, y = smote.fit_resample(X, y)

    # ✅ Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # ✅ Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # ✅ Create DataLoaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=64, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=64, shuffle=False)

    return train_loader, test_loader, X_train_tensor.shape[1]

# ✅ Initialize Training Components
train_loader, test_loader, input_size = load_dataset()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# ✅ Training Loop (10 epochs)
epochs = 200
for epoch in range(epochs):
    model.train()
    total_loss = 0.0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"📢 Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# ✅ Evaluation
model.eval()
total_loss, correct, total = 0.0, 0, 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total if total > 0 else 0.0
print(f"✅ Final Evaluation: Loss={total_loss:.4f}, Accuracy={accuracy:.4f}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# ✅ Load the three datasets
young_adults = pd.read_csv("young_adults.csv")
middle_aged_adults = pd.read_csv("middle_aged_adults.csv")
older_adults = pd.read_csv("older_adults.csv")

# ✅ Separate features and target variable
def prepare_data(df):
    X = df.drop(columns=["TenYearCHD"])
    y = df["TenYearCHD"]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)  # Normalize features
    return X_scaled, y, X.columns  # Return feature names for reference

X_young, y_young, feature_names = prepare_data(young_adults)
X_middle, y_middle, _ = prepare_data(middle_aged_adults)
X_older, y_older, _ = prepare_data(older_adults)

# ✅ Train Random Forest models
def train_rf(X, y):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X, y)
    return model.feature_importances_

importance_young = train_rf(X_young, y_young)
importance_middle = train_rf(X_middle, y_middle)
importance_older = train_rf(X_older, y_older)

# ✅ Combine importance scores into a DataFrame
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Young Adults": importance_young,
    "Middle-Aged Adults": importance_middle,
    "Older Adults": importance_older
}).sort_values(by="Middle-Aged Adults", ascending=False)

# ✅ Plot Feature Importance for Comparison
plt.figure(figsize=(12, 6))
x = np.arange(len(feature_names))
width = 0.25

plt.bar(x - width, feature_importance_df["Young Adults"], width=width, label="Young Adults")
plt.bar(x, feature_importance_df["Middle-Aged Adults"], width=width, label="Middle-Aged Adults")
plt.bar(x + width, feature_importance_df["Older Adults"], width=width, label="Older Adults")

plt.xticks(ticks=x, labels=feature_importance_df["Feature"], rotation=45, ha="right")
plt.xlabel("Feature")
plt.ylabel("Importance Score")
plt.title("Feature Importance Comparison Across Age Groups")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Load your Framingham dataset
df = pd.read_csv("framinghamdataset.csv")  # Change this to your actual dataset path

# Drop any missing values (optional, if needed)
df.dropna(inplace=True)

# Separate features (X) and target (y)
X = df.drop(columns=["TenYearCHD"])  # Features (all columns except target)
y = df["TenYearCHD"]  # Target variable

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Combine back into a DataFrame
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled["TenYearCHD"] = y_resampled

# Shuffle the dataset before splitting
df_resampled = df_resampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Split the dataset into 3 random parts for federated clients
client_1, client_2, client_3 = np.array_split(df_resampled, 3)

# Save each client dataset
client_1.to_csv("client_1.csv", index=False)
client_2.to_csv("client_2.csv", index=False)
client_3.to_csv("client_3.csv", index=False)

print("✅ Dataset successfully upsampled and randomly split into 3 clients!")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Load one client dataset (e.g., client_1.csv)
df = pd.read_csv("client_1.csv")

# Separate features (X) and target (y)
X = df.drop(columns=["TenYearCHD"]).values  # Features
y = df["TenYearCHD"].values  # Target

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Binary classification (2 classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
input_size = X_train.shape[1]
model = MLP(input_size)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch in train_loader:
        X_batch, y_batch = batch
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
    
    train_acc = correct / total * 100
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Train Acc: {train_acc:.2f}%")

# Evaluate on test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        X_batch, y_batch = batch
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

test_acc = correct / total * 100
print(f"\n🔍 Test Accuracy: {test_acc:.2f}%")


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

# Load and preprocess dataset
def load_data(filename):
    df = pd.read_csv(filename)
    
    # Feature selection and scaling
    X = df.drop(columns=["TenYearCHD"]).values
    y = df["TenYearCHD"].values
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    
    # Create DataLoaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    return train_loader, test_loader, X_train.shape[1]

# Define Improved MLP Model
class ImprovedMLP(nn.Module):
    def __init__(self, input_size):
        super(ImprovedMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        self.fc5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        self.fc6 = nn.Linear(64, 2)  # Binary classification

        self.swish = nn.SiLU()  # Swish activation instead of ReLU
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = self.swish(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.swish(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.swish(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.swish(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = self.swish(self.bn5(self.fc5(x)))
        x = self.fc6(x)
        return x

# Training function
def train_model(train_loader, test_loader, input_size, client_id):
    print(f"\n🔹 Training Model for Client {client_id}...")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ImprovedMLP(input_size).to(device)

    # Compute class weights
    class_counts = np.bincount([y for _, y in train_loader.dataset])
    class_weights = torch.tensor([1.0 / class_counts[0], 1.0 / class_counts[1]], dtype=torch.float32).to(device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # Optimizer & Scheduler
    optimizer = optim.Adam(model.parameters(), lr=0.0008, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0001)

    # Training loop
    epochs = 300
    early_stopping_patience = 20
    best_val_acc = 0.0
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in train_loader:
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)
        
        train_acc = correct / total * 100
        scheduler.step()

        # Validation step
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in test_loader:
                X_batch, y_batch = batch
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)

        val_acc = correct / total * 100

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"best_model_client_{client_id}.pth")
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stopping_patience:
            print(f"🛑 Early Stopping at Epoch {epoch + 1}. Best Validation Accuracy: {best_val_acc:.2f}%")
            break

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

    print(f"\n✅ Client {client_id} Best Test Accuracy: {best_val_acc:.2f}%")
    return best_val_acc

# Train on all three datasets
datasets = ["client_1.csv", "client_2.csv", "client_3.csv"]
accuracies = {}

for i, dataset in enumerate(datasets):
    train_loader, test_loader, input_size = load_data(dataset)
    accuracies[f"Client {i+1}"] = train_model(train_loader, test_loader, input_size, i+1)

# Print final results
print("\n🎯 Final Accuracies:")
for client, acc in accuracies.items():
    print(f"{client}: {acc:.2f}%")



🔹 Training Model for Client 1...
Epoch 10/300, Loss: 16.7775, Train Acc: 70.96%, Val Acc: 74.17%
Epoch 20/300, Loss: 15.8957, Train Acc: 72.31%, Val Acc: 72.71%
Epoch 30/300, Loss: 15.1266, Train Acc: 74.56%, Val Acc: 71.46%
Epoch 40/300, Loss: 14.9917, Train Acc: 76.17%, Val Acc: 72.50%
🛑 Early Stopping at Epoch 43. Best Validation Accuracy: 74.38%

✅ Client 1 Best Test Accuracy: 74.38%

🔹 Training Model for Client 2...
Epoch 10/300, Loss: 16.7361, Train Acc: 70.79%, Val Acc: 67.92%
Epoch 20/300, Loss: 15.9323, Train Acc: 73.08%, Val Acc: 70.83%
Epoch 30/300, Loss: 15.1623, Train Acc: 75.01%, Val Acc: 70.21%
Epoch 40/300, Loss: 14.9571, Train Acc: 75.80%, Val Acc: 71.25%
Epoch 50/300, Loss: 14.4939, Train Acc: 75.12%, Val Acc: 71.04%
🛑 Early Stopping at Epoch 55. Best Validation Accuracy: 71.46%

✅ Client 2 Best Test Accuracy: 71.46%

🔹 Training Model for Client 3...
Epoch 10/300, Loss: 16.5689, Train Acc: 71.67%, Val Acc: 73.75%
Epoch 20/300, Loss: 15.3742, Train Acc: 74.54%, Val Ac

In [ ]:
pip install ranger-adabelief

can you reduce the ephos to like 200 700 is too much 
🔹 Training Model for Client 1...
Epoch 10/500, Loss: 33.4631, Train Acc: 74.82%, Val Acc: 71.88%
Epoch 20/500, Loss: 31.3419, Train Acc: 77.89%, Val Acc: 72.08%
Epoch 30/500, Loss: 30.5089, Train Acc: 78.42%, Val Acc: 72.92%
Epoch 40/500, Loss: 29.2711, Train Acc: 79.82%, Val Acc: 72.71%
Epoch 50/500, Loss: 28.1338, Train Acc: 82.27%, Val Acc: 73.12%
Epoch 60/500, Loss: 28.0618, Train Acc: 81.86%, Val Acc: 71.88%
Epoch 70/500, Loss: 27.0236, Train Acc: 82.79%, Val Acc: 73.33%
Epoch 80/500, Loss: 26.1234, Train Acc: 84.72%, Val Acc: 73.12%
Epoch 90/500, Loss: 25.6369, Train Acc: 84.57%, Val Acc: 73.12%
Epoch 100/500, Loss: 24.6136, Train Acc: 86.44%, Val Acc: 73.33%
Epoch 110/500, Loss: 24.8100, Train Acc: 85.82%, Val Acc: 72.71%
Epoch 120/500, Loss: 24.2571, Train Acc: 87.49%, Val Acc: 75.62%
Epoch 130/500, Loss: 23.5207, Train Acc: 87.80%, Val Acc: 73.54%
Epoch 140/500, Loss: 23.3109, Train Acc: 87.80%, Val Acc: 74.17%
Epoch 150/500, Loss: 23.5639, Train Acc: 86.97%, Val Acc: 74.79%
Epoch 160/500, Loss: 22.9443, Train Acc: 88.32%, Val Acc: 73.96%
Epoch 170/500, Loss: 22.4119, Train Acc: 89.10%, Val Acc: 75.21%
Epoch 180/500, Loss: 22.4375, Train Acc: 88.69%, Val Acc: 73.75%
Epoch 190/500, Loss: 21.8623, Train Acc: 90.82%, Val Acc: 75.62%
Epoch 200/500, Loss: 21.0324, Train Acc: 90.77%, Val Acc: 74.58%
Epoch 210/500, Loss: 21.3956, Train Acc: 89.73%, Val Acc: 77.08%
Epoch 220/500, Loss: 20.4825, Train Acc: 91.92%, Val Acc: 76.25%
Epoch 230/500, Loss: 21.1599, Train Acc: 90.56%, Val Acc: 76.04%
Epoch 240/500, Loss: 20.7430, Train Acc: 91.97%, Val Acc: 76.25%
Epoch 250/500, Loss: 20.5461, Train Acc: 91.14%, Val Acc: 76.88%
Epoch 260/500, Loss: 20.5796, Train Acc: 91.40%, Val Acc: 76.88%
Epoch 270/500, Loss: 20.3981, Train Acc: 91.55%, Val Acc: 76.25%
Epoch 280/500, Loss: 20.5405, Train Acc: 91.71%, Val Acc: 78.12%
🛑 Early Stopping at Epoch 283. Best Validation Accuracy: 78.75%

✅ Client 1 Best Test Accuracy: 78.75%

🔹 Training Model for Client 2...
Epoch 10/500, Loss: 33.9601, Train Acc: 72.56%, Val Acc: 69.79%
Epoch 20/500, Loss: 31.9448, Train Acc: 76.73%, Val Acc: 72.08%
Epoch 30/500, Loss: 31.3564, Train Acc: 76.89%, Val Acc: 73.54%
Epoch 40/500, Loss: 30.0252, Train Acc: 79.03%, Val Acc: 72.29%
Epoch 50/500, Loss: 29.3394, Train Acc: 79.50%, Val Acc: 72.08%
Epoch 60/500, Loss: 27.7111, Train Acc: 82.00%, Val Acc: 72.71%
Epoch 70/500, Loss: 26.9407, Train Acc: 84.14%, Val Acc: 73.75%
Epoch 80/500, Loss: 26.7294, Train Acc: 83.20%, Val Acc: 72.71%
Epoch 90/500, Loss: 26.1666, Train Acc: 83.88%, Val Acc: 73.96%
Epoch 100/500, Loss: 25.6589, Train Acc: 85.55%, Val Acc: 73.96%
Epoch 110/500, Loss: 25.5070, Train Acc: 84.82%, Val Acc: 74.17%
Epoch 120/500, Loss: 25.0719, Train Acc: 86.54%, Val Acc: 75.62%
Epoch 130/500, Loss: 24.2863, Train Acc: 86.12%, Val Acc: 75.42%
Epoch 140/500, Loss: 23.8278, Train Acc: 87.85%, Val Acc: 76.04%
Epoch 150/500, Loss: 23.2772, Train Acc: 88.78%, Val Acc: 76.25%
Epoch 160/500, Loss: 23.4531, Train Acc: 87.95%, Val Acc: 75.42%
Epoch 170/500, Loss: 22.3687, Train Acc: 89.57%, Val Acc: 76.88%
Epoch 180/500, Loss: 22.5469, Train Acc: 88.47%, Val Acc: 74.38%
Epoch 190/500, Loss: 22.3148, Train Acc: 89.31%, Val Acc: 75.21%
Epoch 200/500, Loss: 22.4409, Train Acc: 88.94%, Val Acc: 76.46%
Epoch 210/500, Loss: 21.8960, Train Acc: 89.62%, Val Acc: 77.50%
Epoch 220/500, Loss: 21.9468, Train Acc: 90.14%, Val Acc: 76.88%
Epoch 230/500, Loss: 21.6842, Train Acc: 90.71%, Val Acc: 76.67%
Epoch 240/500, Loss: 20.9450, Train Acc: 91.18%, Val Acc: 76.25%
Epoch 250/500, Loss: 20.7201, Train Acc: 90.82%, Val Acc: 76.46%
Epoch 260/500, Loss: 20.9092, Train Acc: 91.24%, Val Acc: 76.46%
Epoch 270/500, Loss: 20.2576, Train Acc: 91.55%, Val Acc: 75.62%
Epoch 280/500, Loss: 20.3992, Train Acc: 91.91%, Val Acc: 76.67%
Epoch 290/500, Loss: 20.1071, Train Acc: 92.44%, Val Acc: 77.50%
🛑 Early Stopping at Epoch 298. Best Validation Accuracy: 78.54%

✅ Client 2 Best Test Accuracy: 78.54%

🔹 Training Model for Client 3...
Epoch 10/500, Loss: 33.5534, Train Acc: 74.54%, Val Acc: 73.54%
Epoch 20/500, Loss: 31.8265, Train Acc: 76.73%, Val Acc: 73.33%
Epoch 30/500, Loss: 30.5611, Train Acc: 78.14%, Val Acc: 73.33%
Epoch 40/500, Loss: 29.5722, Train Acc: 79.86%, Val Acc: 73.75%
Epoch 50/500, Loss: 28.4772, Train Acc: 80.54%, Val Acc: 75.42%
Epoch 60/500, Loss: 27.5022, Train Acc: 82.68%, Val Acc: 74.38%
Epoch 70/500, Loss: 28.0754, Train Acc: 80.49%, Val Acc: 72.71%
Epoch 80/500, Loss: 27.2415, Train Acc: 82.32%, Val Acc: 75.21%
Epoch 90/500, Loss: 26.0268, Train Acc: 85.50%, Val Acc: 76.04%
Epoch 100/500, Loss: 26.2210, Train Acc: 83.78%, Val Acc: 74.79%
Epoch 110/500, Loss: 24.9350, Train Acc: 85.65%, Val Acc: 75.83%
Epoch 120/500, Loss: 24.2676, Train Acc: 87.22%, Val Acc: 75.62%
Epoch 130/500, Loss: 24.4917, Train Acc: 86.70%, Val Acc: 76.04%
🛑 Early Stopping at Epoch 132. Best Validation Accuracy: 77.08%

✅ Client 3 Best Test Accuracy: 77.08%

🎯 Final Accuracies:
Client 1: 78.75%
Client 2: 78.54%
Client 3: 77.08%

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# ✅ Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device: {device}")

# ✅ Updated Neural Network Model
class OptimizedNN(nn.Module):
    def __init__(self, input_size, hidden_size=768, num_classes=2, dropout_rate=0.1):
        super(OptimizedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.drop1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.drop2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        return self.fc3(x)

# ✅ Load Data Function (No Balancing)
def load_data(filename, batch_size=256):
    df = pd.read_csv(filename)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values

    print(f"🔹 Class Distribution: {pd.Series(y).value_counts().to_dict()}")

    # ✅ Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # ✅ Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # ✅ Convert to PyTorch Tensors
    X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
    X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

    # ✅ Dataloaders
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=batch_size, shuffle=False)

    input_size = X_train.shape[1]
    return train_loader, test_loader, input_size

# ✅ Training Function
def train_model(train_loader, test_loader, input_size, num_epochs=100, lr=2e-4, patience=10):
    model = OptimizedNN(input_size).to(device)
    
    # ✅ Apply Label Smoothing to Loss Function
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-5)  
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max", factor=0.5, patience=5)

    best_val_acc = 0.0
    epochs_no_improve = 0
    scaler = torch.amp.GradScaler()  

    for epoch in range(num_epochs):
        model.train()
        train_loss, correct_train = 0.0, 0
        total_train = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()

            with torch.amp.autocast("cuda"):
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  
            scaler.step(optimizer)
            scaler.update()
            optimizer.step()

            train_loss += loss.item() * X_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == y_batch).sum().item()
            total_train += y_batch.size(0)

        train_acc = correct_train / total_train * 100

        # ✅ Validation Phase
        model.eval()
        correct_val, total_val = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == y_batch).sum().item()
                total_val += y_batch.size(0)

        val_acc = correct_val / total_val * 100
        scheduler.step(val_acc)

        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {train_loss/total_train:.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # ✅ Early Stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            epochs_no_improve = 0
            best_model_state = model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"🛑 Early Stopping at Epoch {epoch+1}. Best Validation Accuracy: {best_val_acc:.2f}%")
            break

    # ✅ Load Best Model
    model.load_state_dict(best_model_state)
    print(f"\n✅ Best Test Accuracy: {best_val_acc:.2f}%")
    return best_val_acc

# ✅ Load dataset & Train
train_loader, test_loader, input_size = load_data("client_1.csv", batch_size=256)
train_model(train_loader, test_loader, input_size, num_epochs=100, lr=2e-4)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

# ✅ Load dataset
data_path = "client_1.csv"
df = pd.read_csv(data_path)

# ✅ Assume the last column is the target variable
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# ✅ Feature Selection (Top 10 features)
selector = SelectKBest(score_func=mutual_info_classif, k=10)
X_selected = selector.fit_transform(X, y)

# ✅ Data Split
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Convert to Torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


# ✅ Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Dropout(dropout)]
        for _ in range(num_layers - 1):
            layers += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU(), nn.Dropout(dropout)]
        layers.append(nn.Linear(hidden_dim, 2))  # Output layer for binary classification
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


# ✅ Objective function for Optuna
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    learning_rate = trial.suggest_loguniform("lr", 1e-4, 1e-2)
    hidden_dim = trial.suggest_int("hidden_dim", 32, 256, step=32)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    dropout = trial.suggest_uniform("dropout", 0.1, 0.5)

    # ✅ Data Loaders
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # ✅ Model
    model = MLP(input_dim=X_train.shape[1], hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).to("cuda")
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # ✅ Training Loop
    best_acc = 0
    for epoch in range(50):  # 50 Epochs for tuning
        model.train()
        for batch_x, batch_y in train_loader:
            batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        # ✅ Validation Accuracy
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test_tensor.to("cuda"))
            y_pred = torch.argmax(y_pred, dim=1).cpu().numpy()
            acc = accuracy_score(y_test, y_pred)
            best_acc = max(best_acc, acc)

    return best_acc


# ✅ Run Optuna for Hyperparameter Tuning
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  # 20 Trials for tuning

# ✅ Best Parameters
best_params = study.best_params
print(f"✅ Best Hyperparameters: {best_params}")

# ✅ Train Final Model with Best Hyperparameters
batch_size = best_params["batch_size"]
learning_rate = best_params["lr"]
hidden_dim = best_params["hidden_dim"]
num_layers = best_params["num_layers"]
dropout = best_params["dropout"]

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = MLP(input_dim=X_train.shape[1], hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).to("cuda")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# ✅ Train Full Model
best_acc = 0
for epoch in range(100):  # Full Training
    model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to("cuda"), batch_y.to("cuda")
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # ✅ Validation Accuracy
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor.to("cuda"))
        y_pred = torch.argmax(y_pred, dim=1).cpu().numpy()
        acc = accuracy_score(y_test, y_pred)
        best_acc = max(best_acc, acc)

    print(f"Epoch {epoch+1}/100 | Best Test Acc: {best_acc:.2%}")

print(f"✅ Final Test Accuracy: {best_acc:.2%}")


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from torch.utils.data import Dataset, DataLoader, TensorDataset

# Load dataset
file_path = "client_1.csv"
df = pd.read_csv(file_path)

# Encode categorical labels
label_encoder = LabelEncoder()
df["TenYearCHD"] = label_encoder.fit_transform(df["TenYearCHD"])

# Feature Selection - Select Top 20 Best Features
selector = SelectKBest(f_classif, k=20)
X_selected = selector.fit_transform(df.drop(columns=["TenYearCHD"]), df["TenYearCHD"].values)
y = df["TenYearCHD"].values

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42, stratify=y)

# Standardize Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_dataloaders(batch_size):
    train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
    test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# Define Improved MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))  # Batch Normalization
        layers.append(nn.GELU())  # Improved Activation Function

        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))

        layers.append(nn.Linear(hidden_dim, 2))  # Output layer for binary classification
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# Hyperparameter Tuning with Optuna
def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    learning_rate = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 192, 256])
    num_layers = trial.suggest_int("num_layers", 1, 4)
    dropout = trial.suggest_float("dropout", 0.1, 0.5)

    train_loader, test_loader = create_dataloaders(batch_size)

    model = MLP(input_dim=X_train.shape[1], hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)  # Learning rate scheduler

    best_acc = 0
    for epoch in range(15):  # Tune for 15 epochs per trial
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        scheduler.step()  # Adjust learning rate

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)

        val_acc = correct / total
        best_acc = max(best_acc, val_acc)

    return best_acc

# Run Hyperparameter Optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Increase Trials for Better Results

# Get Best Parameters
best_params = study.best_params
print(f"✅ Best Hyperparameters: {best_params}")

# Train Final Model with Best Params
batch_size = best_params["batch_size"]
learning_rate = best_params["lr"]
hidden_dim = best_params["hidden_dim"]
num_layers = best_params["num_layers"]
dropout = best_params["dropout"]

train_loader, test_loader = create_dataloaders(batch_size)

model = MLP(input_dim=X_train.shape[1], hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).cuda()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)  # Longer decay

best_test_acc = 0

for epoch in range(100):  # Train Final Model for 100 Epochs
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    scheduler.step()  # Adjust Learning Rate

    # Evaluate Model
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    test_acc = correct / total
    best_test_acc = max(best_test_acc, test_acc)
    print(f"Epoch {epoch+1}/100 | Best Test Acc: {best_test_acc:.2%}")

print(f"✅ Final Test Accuracy: {best_test_acc:.2%}")


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader, TensorDataset

# ✅ Load dataset
file_path = "client_1.csv"
df = pd.read_csv(file_path)

# ✅ Ensure target column exists
target_column = "TenYearCHD"
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset.")

# ✅ Separate Features and Target
X = df.drop(columns=[target_column])
y = df[target_column].values

# ✅ Standardize Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ✅ Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# ✅ Create Custom Dataset & DataLoaders
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_dataloaders(batch_size=32):
    train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
    test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# ✅ Define Optimized MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=192, num_layers=2, dropout=0.1155):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))  # Batch Normalization
        layers.append(nn.GELU())  # Improved Activation Function

        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))

        layers.append(nn.Linear(hidden_dim, 2))  # Output Layer for Binary Classification
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# ✅ Train Model using the Best Hyperparameters
def train_and_evaluate(batch_size=32, learning_rate=0.0031, hidden_dim=192, num_layers=2, dropout=0.1155, num_epochs=100):
    train_loader, test_loader = create_dataloaders(batch_size)

    model = MLP(input_dim=X_train.shape[1], hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)  # Adaptive Learning Rate

    best_test_acc = 0

    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        scheduler.step()  # Adjust Learning Rate

        # ✅ Evaluate Model
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)

        test_acc = correct / total
        best_test_acc = max(best_test_acc, test_acc)
        print(f"Epoch {epoch+1}/100 | Best Test Acc: {best_test_acc:.2%}")

    print(f"✅ Final Test Accuracy: {best_test_acc:.2%}")

# ✅ Run Training
train_and_evaluate()


Epoch 1/100 | Best Test Acc: 69.79%
Epoch 2/100 | Best Test Acc: 69.79%
Epoch 3/100 | Best Test Acc: 69.79%
Epoch 4/100 | Best Test Acc: 69.79%
Epoch 5/100 | Best Test Acc: 69.79%
Epoch 6/100 | Best Test Acc: 70.62%
Epoch 7/100 | Best Test Acc: 71.46%
Epoch 8/100 | Best Test Acc: 71.46%
Epoch 9/100 | Best Test Acc: 71.88%
Epoch 10/100 | Best Test Acc: 72.71%
Epoch 11/100 | Best Test Acc: 72.92%
Epoch 12/100 | Best Test Acc: 73.96%
Epoch 13/100 | Best Test Acc: 73.96%
Epoch 14/100 | Best Test Acc: 73.96%
Epoch 15/100 | Best Test Acc: 73.96%
Epoch 16/100 | Best Test Acc: 73.96%
Epoch 17/100 | Best Test Acc: 73.96%
Epoch 18/100 | Best Test Acc: 73.96%
Epoch 19/100 | Best Test Acc: 73.96%
Epoch 20/100 | Best Test Acc: 73.96%
Epoch 21/100 | Best Test Acc: 73.96%
Epoch 22/100 | Best Test Acc: 73.96%
Epoch 23/100 | Best Test Acc: 73.96%
Epoch 24/100 | Best Test Acc: 73.96%
Epoch 25/100 | Best Test Acc: 73.96%
Epoch 26/100 | Best Test Acc: 74.17%
Epoch 27/100 | Best Test Acc: 74.17%
Epoch 28/1

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

# ✅ Load dataset
file_path = "client_1.csv"  # Adjust path if needed
df = pd.read_csv(file_path)

# ✅ Ensure target column exists
target_column = "TenYearCHD"
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset.")

# ✅ Separate Features and Target
X = df.drop(columns=[target_column])
y = df[target_column].values

# ✅ Standardize Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ✅ Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ✅ Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# ✅ Create Custom Dataset & DataLoaders
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_dataloaders(batch_size=64):
    train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
    test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

# ✅ Define Optimized MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=4, dropout=0.2587):
        super(MLP, self).__init__()
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))  # Batch Normalization
        layers.append(nn.GELU())  # Improved Activation Function

        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))

        layers.append(nn.Linear(hidden_dim, 2))  # Output Layer for Binary Classification
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

# ✅ Train Model using the Best Hyperparameters
def train_and_evaluate(batch_size=64, learning_rate=0.00507, hidden_dim=128, num_layers=4, dropout=0.2587, num_epochs=100):
    train_loader, test_loader = create_dataloaders(batch_size)

    model = MLP(input_dim=X_train.shape[1], hidden_dim=hidden_dim, num_layers=num_layers, dropout=dropout).cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)  # Adaptive Learning Rate

    best_test_acc = 0

    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        scheduler.step()  # Adjust Learning Rate

        # ✅ Evaluate Model
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.cuda(), y_batch.cuda()
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)

        test_acc = correct / total
        best_test_acc = max(best_test_acc, test_acc)
        print(f"Epoch {epoch+1}/100 | Best Test Acc: {best_test_acc:.2%}")

    print(f"✅ Final Test Accuracy: {best_test_acc:.2%}")

# ✅ Run Training
train_and_evaluate()


Epoch 1/100 | Best Test Acc: 67.08%
Epoch 2/100 | Best Test Acc: 71.67%
Epoch 3/100 | Best Test Acc: 71.67%
Epoch 4/100 | Best Test Acc: 71.88%
Epoch 5/100 | Best Test Acc: 71.88%
Epoch 6/100 | Best Test Acc: 72.71%
Epoch 7/100 | Best Test Acc: 74.58%
Epoch 8/100 | Best Test Acc: 74.58%
Epoch 9/100 | Best Test Acc: 74.58%
Epoch 10/100 | Best Test Acc: 74.58%
Epoch 11/100 | Best Test Acc: 74.58%
Epoch 12/100 | Best Test Acc: 74.58%
Epoch 13/100 | Best Test Acc: 74.58%
Epoch 14/100 | Best Test Acc: 75.21%
Epoch 15/100 | Best Test Acc: 75.42%
Epoch 16/100 | Best Test Acc: 75.42%
Epoch 17/100 | Best Test Acc: 75.42%
Epoch 18/100 | Best Test Acc: 75.83%
Epoch 19/100 | Best Test Acc: 75.83%
Epoch 20/100 | Best Test Acc: 75.83%
Epoch 21/100 | Best Test Acc: 75.83%
Epoch 22/100 | Best Test Acc: 75.83%
Epoch 23/100 | Best Test Acc: 75.83%
Epoch 24/100 | Best Test Acc: 75.83%
Epoch 25/100 | Best Test Acc: 75.83%
Epoch 26/100 | Best Test Acc: 75.83%
Epoch 27/100 | Best Test Acc: 75.83%
Epoch 28/1

In [ ]:
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU only'}")


model 2

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from typing import List, Dict, Tuple

# ✅ Set device for computation (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load dataset
file_path = "client_2.csv"  # Adjust path if needed
df = pd.read_csv(file_path)

# ✅ Ensure target column exists
target_column = "TenYearCHD"
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset.")

# ✅ Separate Features and Target
X = df.drop(columns=[target_column]).values
y = df[target_column].values

# ✅ Standardize Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ✅ Train/Test Split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# ✅ Create Custom Dataset & DataLoaders
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_dataloaders(batch_size=64):
    train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
    test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

train_loader, test_loader = create_dataloaders()

# ✅ Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=4, dropout=0.2587):
        super(MLP, self).__init__()
        layers = [
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU()
        ]
        for _ in range(num_layers - 1):
            layers.extend([
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout)
            ])
        layers.append(nn.Linear(hidden_dim, 2))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# ✅ Initialize model
input_dim = X_train.shape[1]
model = MLP(input_dim=input_dim).to(device)

# ✅ Define optimizer, loss function, and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.00507)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25)

# ✅ Fix Parameter Handling
def get_parameters(model: nn.Module) -> List[np.ndarray]:
    return [val.cpu().detach().numpy() for val in model.state_dict().values()]

def set_parameters(model: nn.Module, parameters: List[np.ndarray]) -> None:
    state_dict = model.state_dict()
    new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
    model.load_state_dict(new_state_dict, strict=True)

# ✅ Define the Federated Learning Client
class FLClient(fl.client.NumPyClient):
    def get_parameters(self, config: Dict[str, str]) -> List[np.ndarray]:
        return get_parameters(model)

    def set_parameters(self, parameters: List[np.ndarray]) -> None:
        set_parameters(model, parameters)

    def fit(self, parameters, config):
        """Train the local model and return updated parameters."""
        print("📤 Client received parameters for training.")
        self.set_parameters(parameters)
        model.train()
        
        local_epochs = 50  # Ensure sufficient local training
        total_loss = 0.0
        for epoch in range(local_epochs):
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"✅ Client training complete. Avg Loss: {avg_loss:.4f}")

        # ✅ **Return three values to avoid ValueError**
        return self.get_parameters(config), len(train_loader.dataset), {"loss": avg_loss}

    def evaluate(self, parameters: List[np.ndarray], config: Dict[str, str]) -> Tuple[float, int, Dict]:
        """Evaluate the model."""
        self.set_parameters(parameters)
        model.eval()
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)

        accuracy = correct / total if total > 0 else 0.0
        avg_loss = total_loss / len(test_loader)
        print(f"✅ Evaluation: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}")

        # ✅ **Return three values (loss, dataset size, accuracy)**
        return float(avg_loss), len(test_loader.dataset), {"accuracy": accuracy}

# ✅ Start the Federated Learning Client
fl.client.start_client(
    server_address="127.0.0.1:8081",
    client=FLClient().to_client(),  # Use .to_client() to avoid deprecation warning
)


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8081: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:8081: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.\r\n -- 10061)", grpc_status:14, created_time:"2025-03-17T20:25:53.3790492+00:00"}"
>

testing 
client and server (my text)


In [11]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

# Define the same neural network model as on the server
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(15, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Training function
def train(net, trainloader, epochs=20):
    net.train()
    optimizer = optim.Adam(net.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        for data, target in trainloader:
            optimizer.zero_grad()
            output = net(data)
            loss = loss_fn(output.squeeze(), target)
            loss.backward()
            optimizer.step()

# Testing function
def test(net, valloader):
    net.eval()
    correct = 0
    total = 0
    loss_fn = nn.BCELoss()
    
    with torch.no_grad():
        for data, target in valloader:
            output = net(data)
            loss = loss_fn(output.squeeze(), target)
            predicted = (output.squeeze() > 0.5).float()
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = correct / total
    print(f"Validation Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")
    return loss.item(), accuracy

# Define dataset
class HeartDiseaseDataset(Dataset):
    def __init__(self, data, target):
        self.data = torch.tensor(data.values, dtype=torch.float32)
        self.target = torch.tensor(target.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

# Load and preprocess data
data = pd.read_csv('client_2.csv')  # Change file name for different clients
X = data.drop(columns=["TenYearCHD"])
y = data["TenYearCHD"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
train_dataset = HeartDiseaseDataset(pd.DataFrame(X_train), pd.Series(y_train))
test_dataset = HeartDiseaseDataset(pd.DataFrame(X_test), pd.Series(y_test))

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the Flower Client
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, pid, net, trainloader, valloader):
        self.pid = pid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.pid}] Sending model parameters to server.")
        return [val.detach().cpu().numpy() for val in self.net.parameters()]

    def fit(self, parameters, config):
        server_round = config.get("server_round", 1)  # Prevent KeyError
        local_epochs = config.get("local_epochs", 60)  # Train for 20 epochs before sending updates

        print(f"[Client {self.pid}, round {server_round}] Training for {local_epochs} epochs...")
        
        set_parameters(self.net, parameters)  # Load global model parameters
        train(self.net, self.trainloader, epochs=local_epochs)  # Train for 20 epochs
        return self.get_parameters(config), len(self.trainloader.dataset), {}


    def evaluate(self, parameters, config):
        print(f"[Client {self.pid}] Evaluating model...")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return loss, len(self.valloader.dataset), {"accuracy": accuracy}

# Helper function to set model parameters
def set_parameters(net, parameters):
    for param, new_param in zip(net.parameters(), parameters):
        param.data = torch.tensor(new_param)
        
def train(net, trainloader, epochs):
    net.train()
    optimizer = optim.Adam(net.parameters(), lr=0.0005)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        epoch_loss = 0
        for data, target in trainloader:
            optimizer.zero_grad()
            output = net(data)
            loss = loss_fn(output.squeeze(), target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(trainloader):.4f}")


if __name__ == "__main__":
    model = SimpleNN()
    client = FlowerClient(pid=1, net=model, trainloader=trainloader, valloader=valloader)

    fl.client.start_client(
        server_address="127.0.0.1:8081",
        client=client.to_client()  # Updated to latest Flower API
    )


	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: train message afa2fb4a-981f-469a-bd0c-e6b5379dba56


[Client 1, round 1] Training for 60 epochs...
Epoch 1/60, Loss: 0.6858
Epoch 2/60, Loss: 0.6588
Epoch 3/60, Loss: 0.6374
Epoch 4/60, Loss: 0.6217
Epoch 5/60, Loss: 0.6101
Epoch 6/60, Loss: 0.6004
Epoch 7/60, Loss: 0.5943
Epoch 8/60, Loss: 0.5869
Epoch 9/60, Loss: 0.5805
Epoch 10/60, Loss: 0.5756
Epoch 11/60, Loss: 0.5714
Epoch 12/60, Loss: 0.5677
Epoch 13/60, Loss: 0.5634
Epoch 14/60, Loss: 0.5582
Epoch 15/60, Loss: 0.5550
Epoch 16/60, Loss: 0.5502
Epoch 17/60, Loss: 0.5466
Epoch 18/60, Loss: 0.5438
Epoch 19/60, Loss: 0.5381
Epoch 20/60, Loss: 0.5355
Epoch 21/60, Loss: 0.5313
Epoch 22/60, Loss: 0.5279
Epoch 23/60, Loss: 0.5251
Epoch 24/60, Loss: 0.5191
Epoch 25/60, Loss: 0.5172
Epoch 26/60, Loss: 0.5145
Epoch 27/60, Loss: 0.5088
Epoch 28/60, Loss: 0.5076
Epoch 29/60, Loss: 0.5031
Epoch 30/60, Loss: 0.4991
Epoch 31/60, Loss: 0.4947
Epoch 32/60, Loss: 0.4948
Epoch 33/60, Loss: 0.4946
Epoch 34/60, Loss: 0.4854
Epoch 35/60, Loss: 0.4838
Epoch 36/60, Loss: 0.4795
Epoch 37/60, Loss: 0.4766
E

INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 058071b4-aa43-45f8-be5f-9736e3385d5d
INFO :      Sent reply
INFO :      
INFO :      Received: train message 7ff15dc3-cbe9-4b82-a1af-13c5c4c34181


Epoch 60/60, Loss: 0.4148
[Client 1] Sending model parameters to server.
[Client 1] Evaluating model...
Validation Loss: 0.7239, Accuracy: 0.7042
[Client 1, round 1] Training for 60 epochs...
Epoch 1/60, Loss: 0.4986
Epoch 2/60, Loss: 0.4834
Epoch 3/60, Loss: 0.4779
Epoch 4/60, Loss: 0.4704
Epoch 5/60, Loss: 0.4671
Epoch 6/60, Loss: 0.4634
Epoch 7/60, Loss: 0.4563
Epoch 8/60, Loss: 0.4537
Epoch 9/60, Loss: 0.4488
Epoch 10/60, Loss: 0.4462
Epoch 11/60, Loss: 0.4427
Epoch 12/60, Loss: 0.4383
Epoch 13/60, Loss: 0.4344
Epoch 14/60, Loss: 0.4332
Epoch 15/60, Loss: 0.4302
Epoch 16/60, Loss: 0.4281
Epoch 17/60, Loss: 0.4243
Epoch 18/60, Loss: 0.4205
Epoch 19/60, Loss: 0.4172
Epoch 20/60, Loss: 0.4143
Epoch 21/60, Loss: 0.4130
Epoch 22/60, Loss: 0.4105
Epoch 23/60, Loss: 0.4067
Epoch 24/60, Loss: 0.4044
Epoch 25/60, Loss: 0.3998
Epoch 26/60, Loss: 0.3990
Epoch 27/60, Loss: 0.3944
Epoch 28/60, Loss: 0.3928
Epoch 29/60, Loss: 0.3897
Epoch 30/60, Loss: 0.3845
Epoch 31/60, Loss: 0.3872
Epoch 32/60

INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 5d1db0ac-7b2d-4958-946a-e033d76d0ca2
INFO :      Sent reply
INFO :      
INFO :      Received: train message 5959ca36-1350-43af-8b34-cd6ea9427d64


...

Epoch 60/60, Loss: 0.2493
[Client 1] Sending model parameters to server.
[Client 1] Evaluating model...
Validation Loss: 0.9101, Accuracy: 0.7292
[Client 1, round 1] Training for 60 epochs...
Epoch 1/60, Loss: 0.3480
Epoch 2/60, Loss: 0.3298
Epoch 3/60, Loss: 0.3212
Epoch 4/60, Loss: 0.3133
Epoch 5/60, Loss: 0.3054
Epoch 6/60, Loss: 0.3033
Epoch 7/60, Loss: 0.2952
Epoch 8/60, Loss: 0.2955
Epoch 9/60, Loss: 0.2873
Epoch 10/60, Loss: 0.2805
Epoch 11/60, Loss: 0.2805
Epoch 12/60, Loss: 0.2764
Epoch 13/60, Loss: 0.2704
Epoch 14/60, Loss: 0.2720
Epoch 15/60, Loss: 0.2671
Epoch 16/60, Loss: 0.2661
Epoch 17/60, Loss: 0.2677
Epoch 18/60, Loss: 0.2617
Epoch 19/60, Loss: 0.2591
Epoch 20/60, Loss: 0.2564
Epoch 21/60, Loss: 0.2526
Epoch 22/60, Loss: 0.2524
Epoch 23/60, Loss: 0.2497
Epoch 24/60, Loss: 0.2493
Epoch 25/60, Loss: 0.2468
Epoch 26/60, Loss: 0.2456
Epoch 27/60, Loss: 0.2405
Epoch 28/60, Loss: 0.2422
Epoch 29/60, Loss: 0.2376
Epoch 30/60, Loss: 0.2376
Epoch 31/60, Loss: 0.2365
Epoch 32/60

INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 14232d5b-7b60-411e-a80f-65b1571f6c7b
INFO :      Sent reply
INFO :      
INFO :      Received: train message ef40f4c5-7747-49c9-8203-9bbc654d0eaf


Epoch 60/60, Loss: 0.1984
[Client 1] Sending model parameters to server.
[Client 1] Evaluating model...
Validation Loss: 1.0748, Accuracy: 0.7812
[Client 1, round 1] Training for 60 epochs...
Epoch 1/60, Loss: 0.3095
Epoch 2/60, Loss: 0.2872
Epoch 3/60, Loss: 0.2738
Epoch 4/60, Loss: 0.2650
Epoch 5/60, Loss: 0.2576
Epoch 6/60, Loss: 0.2507
Epoch 7/60, Loss: 0.2448
Epoch 8/60, Loss: 0.2396
Epoch 9/60, Loss: 0.2385
Epoch 10/60, Loss: 0.2323
Epoch 11/60, Loss: 0.2346
Epoch 12/60, Loss: 0.2253
Epoch 13/60, Loss: 0.2219
Epoch 14/60, Loss: 0.2206
Epoch 15/60, Loss: 0.2198
Epoch 16/60, Loss: 0.2170
Epoch 17/60, Loss: 0.2120
Epoch 18/60, Loss: 0.2106
Epoch 19/60, Loss: 0.2118
Epoch 20/60, Loss: 0.2059
Epoch 21/60, Loss: 0.2042
Epoch 22/60, Loss: 0.2002
Epoch 23/60, Loss: 0.1989
Epoch 24/60, Loss: 0.2030
Epoch 25/60, Loss: 0.1987
Epoch 26/60, Loss: 0.1956
Epoch 27/60, Loss: 0.1977
Epoch 28/60, Loss: 0.1950
Epoch 29/60, Loss: 0.1926
Epoch 30/60, Loss: 0.1899
Epoch 31/60, Loss: 0.1896
Epoch 32/60

INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 98d3577a-5c60-4c7e-87f2-7138ea327385
INFO :      Sent reply
INFO :      
INFO :      Received: reconnect message c1a13cc0-e63c-45e2-bf8c-8093c7829368
INFO :      Disconnect and shut down


Epoch 60/60, Loss: 0.1548
[Client 1] Sending model parameters to server.
[Client 1] Evaluating model...
Validation Loss: 1.0202, Accuracy: 0.7604
